In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType

In [2]:
spark = SparkSession.builder.appName("amount-spent-df").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/18 22:47:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
schema = StructType([
    StructField("customerId", IntegerType(), False), \
    StructField("itemId", IntegerType(), False), \
    StructField("amount-spent", FloatType(), True)
])

In [6]:
df = spark.read.schema(schema).csv("customer-orders.csv")
df.show()
df.printSchema()

+----------+------+------------+
|customerId|itemId|amount-spent|
+----------+------+------------+
|        44|  8602|       37.19|
|        35|  5368|       65.89|
|         2|  3391|       40.64|
|        47|  6694|       14.98|
|        29|   680|       13.08|
|        91|  8900|       24.59|
|        70|  3959|       68.68|
|        85|  1733|       28.53|
|        53|  9900|       83.55|
|        14|  1505|        4.32|
|        51|  3378|        19.8|
|        42|  6926|       57.77|
|         2|  4424|       55.77|
|        79|  9291|       33.17|
|        50|  3901|       23.57|
|        20|  6633|        6.49|
|        15|  6148|       65.53|
|        44|  8331|       99.19|
|         5|  3505|       64.18|
|        48|  5539|       32.42|
+----------+------+------------+
only showing top 20 rows

root
 |-- customerId: integer (nullable = true)
 |-- itemId: integer (nullable = true)
 |-- amount-spent: float (nullable = true)



In [12]:
sumAmount = df.groupBy("customerId").agg(func.round(func.sum("amount-spent"), 2).alias("total-spent"))
sumAmount.show()

+----------+-----------+
|customerId|total-spent|
+----------+-----------+
|        31|    4765.05|
|        85|    5503.43|
|        65|    5140.35|
|        53|     4945.3|
|        78|    4524.51|
|        34|     5330.8|
|        81|    5112.71|
|        28|    5000.71|
|        76|    4904.21|
|        27|    4915.89|
|        26|     5250.4|
|        44|    4756.89|
|        12|    4664.59|
|        91|    4642.26|
|        22|    5019.45|
|        93|    5265.75|
|        47|     4316.3|
|         1|     4958.6|
|        52|    5245.06|
|        13|    4367.62|
+----------+-----------+
only showing top 20 rows



In [13]:
totalByCustomerSorted = sumAmount.sort("total-spent")

In [14]:
totalByCustomerSorted.show(totalByCustomerSorted.count())

+----------+-----------+
|customerId|total-spent|
+----------+-----------+
|        45|    3309.38|
|        79|    3790.57|
|        96|    3924.23|
|        23|    4042.65|
|        99|    4172.29|
|        75|     4178.5|
|        36|    4278.05|
|        98|    4297.26|
|        47|     4316.3|
|        77|    4327.73|
|        13|    4367.62|
|        48|    4384.33|
|        49|     4394.6|
|        94|    4475.57|
|        67|    4505.79|
|        50|    4517.27|
|        78|    4524.51|
|         5|    4561.07|
|        57|     4628.4|
|        83|     4635.8|
|        91|    4642.26|
|        74|    4647.13|
|        84|    4652.94|
|         3|    4659.63|
|        12|    4664.59|
|        66|    4681.92|
|        56|    4701.02|
|        21|    4707.41|
|        80|    4727.86|
|        14|    4735.03|
|        37|     4735.2|
|         7|    4755.07|
|        44|    4756.89|
|        31|    4765.05|
|        82|    4812.49|
|         4|    4815.05|
|        10|     4819.7|


In [15]:
spark.stop()